In [1]:
from allensdk.brain_observatory.behavior.behavior_ophys_session import BehaviorOphysSession
import visual_behavior.database as db
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## get alternate IDs for this mouse

In [2]:
labtracks_id = 435431

In [3]:
ids = db.get_mouse_ids('labtracks_id', labtracks_id).iloc[0]
ids

donor_id        807248984
labtracks_id       435431
specimen_id     807248992
Name: 0, dtype: object

## lims query to get all experiments for this session
join with sessions to allow for a search on specimen_id

In [4]:
query = '''
select 
    oe.id as ophys_experiment_id, 
    os.id as ophys_session_id,
    oe.workflow_state,
    os.specimen_id,
    os.date_of_acquisition,
    os.stimulus_name
from ophys_experiments as oe
join ophys_sessions as os on oe.ophys_session_id=os.id
where specimen_id = {}
'''
mouse_experiments = db.lims_query(query.format(ids['specimen_id']))
mouse_experiments

,ophys_experiment_id,ophys_session_id,workflow_state,specimen_id,date_of_acquisition,stimulus_name
0,889806749,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
1,889806747,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
2,889806745,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
3,889806743,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
4,889806741,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
5,889806739,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
6,889806737,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
7,889806735,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
8,881949081,881094781,passed,807248992,2019-06-05 09:12:12.735423,OPHYS_2_images_A_passive
9,881949079,881094781,passed,807248992,2019-06-05 09:12:12.735423,OPHYS_2_images_A_passive


## for every experiment, open the SDK session object, concatenate the cell_specimen_tables

In [5]:
cell_table = []
for idx,row in mouse_experiments.iterrows():
    if row['workflow_state'] == 'passed':
        session = BehaviorOphysSession.from_lims(row['ophys_experiment_id'])
        ct = session.cell_specimen_table.reset_index().copy()
        for column in mouse_experiments.columns:
            ct[column] = row[column]
        cell_table.append(ct)
cell_table = pd.concat(cell_table).reset_index()

In [6]:
cell_table

,index,cell_specimen_id,cell_roi_id,height,image_mask,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,ophys_experiment_id,ophys_session_id,workflow_state,specimen_id,date_of_acquisition,stimulus_name
0,0,1063220336,889854795,11,"[[False, False, False, False, False, False, Fa...",0,3.38914,4.30913,8.17159,3.69066,False,18,436,176,889806749,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
1,1,1018095208,889854799,33,"[[False, False, False, False, False, False, Fa...",0,3.38914,4.30913,8.17159,3.69066,True,30,352,183,889806749,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
2,2,1018095263,889854803,12,"[[False, False, False, False, False, False, Fa...",0,3.38914,4.30913,8.17159,3.69066,True,14,42,130,889806749,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
3,3,1018095406,889854808,35,"[[False, False, False, False, False, False, Fa...",0,3.38914,4.30913,8.17159,3.69066,True,17,479,130,889806749,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
4,4,1018095266,889854815,13,"[[False, False, False, False, False, False, Fa...",0,3.38914,4.30913,8.17159,3.69066,True,18,80,110,889806749,889042121,passed,807248992,2019-06-18 09:45:02.338128,OPHYS_7_receptive_field_mapping
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,32,1063220470,891166680,14,"[[False, False, False, False, False, False, Fa...",0,9.68738,7.53011,6.80665,11.43710,False,14,410,70,891108758,889944877,passed,807248992,2019-06-19 09:25:00.659441,OPHYS_5_images_B_passive
3116,33,1063220413,891166682,12,"[[False, False, False, False, False, False, Fa...",0,9.68738,7.53011,6.80665,11.43710,True,14,292,69,891108758,889944877,passed,807248992,2019-06-19 09:25:00.659441,OPHYS_5_images_B_passive
3117,34,1063220669,891166684,14,"[[False, False, False, False, False, False, Fa...",0,9.68738,7.53011,6.80665,11.43710,False,12,98,191,891108758,889944877,passed,807248992,2019-06-19 09:25:00.659441,OPHYS_5_images_B_passive
3118,35,1063220463,891166686,19,"[[False, False, False, False, False, False, Fa...",0,9.68738,7.53011,6.80665,11.43710,True,27,375,142,891108758,889944877,passed,807248992,2019-06-19 09:25:00.659441,OPHYS_5_images_B_passive


## Are there any duplicate cell_specimen_ids within a given session?

In [7]:
cell_table.duplicated(subset=['cell_specimen_id', 'ophys_session_id']).value_counts()

False    3120
dtype: int64

## No! Success!

## what about cells that are duplicated, but not within the same session?

In [8]:
cell_table.duplicated(subset=['cell_specimen_id']).value_counts()

True     1940
False    1180
dtype: int64